In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, roc_curve, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.decomposition import PCA
import mrmr
import optuna
import torch
import gpytorch
import torchmetrics

data_dir = 'data'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [47]:
data_dir = 'data'

df = pd.read_csv(os.path.join(data_dir, 'final_data_log_targets_with_demographic.csv'))


df['tau_ab_ratio'] = np.exp(df['TAU_bl']) / df['ABETA_bl']
df['ptau_ab_ratio'] =np.exp(df['PTAU_bl']) / df['ABETA_bl']

df['A+'] = df['ABETA_bl'].apply(lambda x: 1 if x < 880 else 0)
df['T+'] = df['ptau_ab_ratio'].apply(lambda x: 1 if x > 0.028 else 0)
df['N+'] = df['tau_ab_ratio'].apply(lambda x: 1 if x > 0.33 else 0)

df =df.drop(columns=['DX_bl','DX','AGE','PTGENDER','PTRACCAT','FHQMOM','FHQMOMAD','FHQDAD','FHQDADAD','RAVLT_perc_forgetting_bl','PTEDUCAT','TAU_bl','PTAU_bl','ABETA_bl', 'MMSE_bl','EcogPtTotal_bl', 'LDELTOTAL_BL', 'mPACCdigit_bl', 'mPACCtrailsB_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl','tau_ab_ratio',	'ptau_ab_ratio'])
df

,SUVr_ACC_pre_L.nii,SUVr_ACC_pre_R.nii,SUVr_ACC_sub_L.nii,SUVr_ACC_sub_R.nii,SUVr_ACC_sup_L.nii,SUVr_ACC_sup_R.nii,SUVr_Amygdala_L.nii,SUVr_Amygdala_R.nii,SUVr_Angular_L.nii,SUVr_Angular_R.nii,...,SUVr_Vermis_6.nii,SUVr_Vermis_7.nii,SUVr_Vermis_8.nii,SUVr_Vermis_9.nii,SUVr_Vermis_10.nii,SUVr_VTA_L.nii,SUVr_VTA_R.nii,A+,T+,N+
0,-1.031439,-0.939867,-0.941345,-0.953400,0.039320,0.038841,0.011054,0.014155,-1.532557,-1.429775,...,0.018253,0.009289,0.011917,0.008881,0.006231,0.000712,0.000659,1,0,0
1,1.190475,1.118614,1.144220,0.928409,0.057261,0.053242,0.013008,0.017184,1.740331,2.026532,...,0.016903,0.009108,0.012666,0.008033,0.006029,0.000770,0.000699,1,1,1
2,1.481382,1.476620,1.518520,1.375084,0.054272,0.053113,0.013620,0.016730,1.590446,1.179038,...,0.019224,0.010413,0.012902,0.007799,0.005730,0.000804,0.000760,1,1,1
3,-0.074118,-0.114674,-0.401073,-0.094141,0.044834,0.042373,0.011938,0.015462,-0.673442,-0.323968,...,0.019834,0.010011,0.012782,0.009789,0.006613,0.000748,0.000675,1,0,0
4,-0.174385,-0.456932,-0.096572,-0.210932,0.042674,0.039067,0.011457,0.013684,-0.166624,-0.560099,...,0.017897,0.009361,0.012595,0.008533,0.005170,0.000675,0.000675,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,0.152440,0.187233,-0.566841,-0.202246,0.042117,0.045545,0.013725,0.016478,-1.175662,-1.357875,...,0.018864,0.008398,0.011478,0.009674,0.007555,0.000818,0.000764,0,0,0
534,-0.650453,-0.313642,-0.686054,-0.794358,0.040523,0.041999,0.011000,0.014486,0.058629,0.597411,...,0.016943,0.008936,0.011930,0.008579,0.006243,0.000678,0.000581,0,0,0
535,-0.461427,-0.219473,-0.177314,-0.159231,0.046125,0.047444,0.010035,0.012790,-0.120657,-0.074618,...,0.015693,0.008374,0.011329,0.008016,0.006976,0.000753,0.000723,0,0,0
536,-1.544436,-1.012396,-1.230629,-1.135321,0.035993,0.039143,0.010660,0.014074,-0.860438,-0.772415,...,0.017387,0.008620,0.010525,0.007063,0.005713,0.000815,0.000768,0,0,0


# STL

In [48]:
class SVM(torch.nn.Module):
    def __init__(self, features, targets, kernel: gpytorch.kernels.Kernel):
        super().__init__()
        self.features = features
        self.targets = targets
        self.kernel = kernel
        self.w = torch.nn.Parameter(torch.randn(features.shape[1], 1), requires_grad=True)
        self.b = torch.nn.Parameter(torch.randn(1), requires_grad=True)
        self.classifier = torch.nn.Sigmoid()
        
    def forward(self, x):
        plane_output = x.matmul(self.w) + self.b
        return self.classifier(plane_output)

In [49]:
X = df.drop(columns=['A+', 'T+', 'N+'])
y = df['A+']

train_acc = []
train_pre = []
train_rec = []
train_f1 = []
train_auroc = []

test_acc = []
test_pre = []
test_rec = []
test_f1 = []
test_auroc = []

for i in range(0, 3):
    rs = np.random.randint(0, 75)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=rs)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device).unsqueeze(1)
    y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device).unsqueeze(1)

    model = SVM(X_train, y_train, kernel=gpytorch.kernels.RBFKernel()).to(device)
    bhl = torchmetrics.classification.BinaryHingeLoss(squared=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-4)

    # Training
    for epoch in range(1, 101):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = bhl(output, y_train)
        loss.backward()
        optimizer.step()

    # EVAL: Train
    model.eval()
    with torch.no_grad():
        y_pred_train = model(X_train)
        y_pred_train_labels = (y_pred_train > 0.5).float()
        y_pred_train_np = y_pred_train_labels.cpu().numpy()
        y_train_np = y_train.cpu().numpy()
        y_pred_train_proba = y_pred_train.cpu().numpy()

        train_acc.append(accuracy_score(y_train_np, y_pred_train_np))
        train_pre.append(precision_score(y_train_np, y_pred_train_np))
        train_rec.append(recall_score(y_train_np, y_pred_train_np))
        train_f1.append(f1_score(y_train_np, y_pred_train_np))
        train_auroc.append(roc_auc_score(y_train_np, y_pred_train_proba))

    # EVAL: Test
    with torch.no_grad():
        y_pred_test = model(X_test)
        y_pred_test_labels = (y_pred_test > 0.5).float()
        y_pred_test_np = y_pred_test_labels.cpu().numpy()
        y_test_np = y_test.cpu().numpy()
        y_pred_test_proba = y_pred_test.cpu().numpy()

        test_acc.append(accuracy_score(y_test_np, y_pred_test_np))
        test_pre.append(precision_score(y_test_np, y_pred_test_np))
        test_rec.append(recall_score(y_test_np, y_pred_test_np))
        test_f1.append(f1_score(y_test_np, y_pred_test_np))
        test_auroc.append(roc_auc_score(y_test_np, y_pred_test_proba))


print("=== Train Metrics ===")
print("Accuracy: ", np.mean(train_acc), "±", np.std(train_acc))
print("Precision:", np.mean(train_pre), "±", np.std(train_pre))
print("Recall:   ", np.mean(train_rec), "±", np.std(train_rec))
print("F1 Score: ", np.mean(train_f1), "±", np.std(train_f1))
print("AUROC:    ", np.mean(train_auroc), "±", np.std(train_auroc))

print("\n=== Test Metrics ===")
print("Accuracy: ", np.mean(test_acc), "±", np.std(test_acc))
print("Precision:", np.mean(test_pre), "±", np.std(test_pre))
print("Recall:   ", np.mean(test_rec), "±", np.std(test_rec))
print("F1 Score: ", np.mean(test_f1), "±", np.std(test_f1))
print("AUROC:    ", np.mean(test_auroc), "±", np.std(test_auroc))


=== Train Metrics ===
Accuracy:  0.848062015503876 ± 0.009744034953470197
Precision: 0.8650804045307011 ± 0.016247478941485406
Recall:    0.8212431390001482 ± 0.00912297395728929
F1 Score:  0.8424841616770483 ± 0.00882714149336615
AUROC:     0.8843371221486486 ± 0.007261757046355939

=== Test Metrics ===
Accuracy:  0.8703703703703702 ± 0.015120307054217125
Precision: 0.8690476190476191 ± 0.01151081052195653
Recall:    0.86996336996337 ± 0.04162387610755868
F1 Score:  0.8687084520417855 ± 0.01530188323297435
AUROC:     0.9075091575091575 ± 0.012358131979880622


In [50]:
X = df.drop(columns=['A+', 'T+', 'N+'])
y = df['T+']

train_acc = []
train_pre = []
train_rec = []
train_f1 = []
train_auroc = []

test_acc = []
test_pre = []
test_rec = []
test_f1 = []
test_auroc = []

for i in range(0, 3):
    rs = np.random.randint(0, 75)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=rs)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device).unsqueeze(1)
    y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device).unsqueeze(1)

    model = SVM(X_train, y_train, kernel=gpytorch.kernels.RBFKernel()).to(device)
    bhl = torchmetrics.classification.BinaryHingeLoss(squared=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-4)

    # Training
    for epoch in range(1, 101):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = bhl(output, y_train)
        loss.backward()
        optimizer.step()

    # EVAL: Train
    model.eval()
    with torch.no_grad():
        y_pred_train = model(X_train)
        y_pred_train_labels = (y_pred_train > 0.5).float()
        y_pred_train_np = y_pred_train_labels.cpu().numpy()
        y_train_np = y_train.cpu().numpy()
        y_pred_train_proba = y_pred_train.cpu().numpy()

        train_acc.append(accuracy_score(y_train_np, y_pred_train_np))
        train_pre.append(precision_score(y_train_np, y_pred_train_np))
        train_rec.append(recall_score(y_train_np, y_pred_train_np))
        train_f1.append(f1_score(y_train_np, y_pred_train_np))
        train_auroc.append(roc_auc_score(y_train_np, y_pred_train_proba))

    # EVAL: Test
    with torch.no_grad():
        y_pred_test = model(X_test)
        y_pred_test_labels = (y_pred_test > 0.5).float()
        y_pred_test_np = y_pred_test_labels.cpu().numpy()
        y_test_np = y_test.cpu().numpy()
        y_pred_test_proba = y_pred_test.cpu().numpy()

        test_acc.append(accuracy_score(y_test_np, y_pred_test_np))
        test_pre.append(precision_score(y_test_np, y_pred_test_np))
        test_rec.append(recall_score(y_test_np, y_pred_test_np))
        test_f1.append(f1_score(y_test_np, y_pred_test_np))
        test_auroc.append(roc_auc_score(y_test_np, y_pred_test_proba))

print("=== Train Metrics ===")
print("Accuracy: ", np.mean(train_acc), "±", np.std(train_acc))
print("Precision:", np.mean(train_pre), "±", np.std(train_pre))
print("Recall:   ", np.mean(train_rec), "±", np.std(train_rec))
print("F1 Score: ", np.mean(train_f1), "±", np.std(train_f1))
print("AUROC:    ", np.mean(train_auroc), "±", np.std(train_auroc))

print("\n=== Test Metrics ===")
print("Accuracy: ", np.mean(test_acc), "±", np.std(test_acc))
print("Precision:", np.mean(test_pre), "±", np.std(test_pre))
print("Recall:   ", np.mean(test_rec), "±", np.std(test_rec))
print("F1 Score: ", np.mean(test_f1), "±", np.std(test_f1))
print("AUROC:    ", np.mean(test_auroc), "±", np.std(test_auroc))

=== Train Metrics ===
Accuracy:  0.9224806201550387 ± 0.016371094637164997
Precision: 0.9270652436097683 ± 0.011486234609613854
Recall:    0.909160718834694 ± 0.024589684276704063
F1 Score:  0.9179762038869878 ± 0.018134298707607563
AUROC:     0.9595148728692062 ± 0.00031658878023908864

=== Test Metrics ===
Accuracy:  0.8950617283950617 ± 0.01573771454812586
Precision: 0.8845707751368129 ± 0.02030793157185032
Recall:    0.9068984474644853 ± 0.028419574362720825
F1 Score:  0.8951702281034356 ± 0.01502310853215183
AUROC:     0.9267357053324035 ± 0.011021385819993065


In [51]:
X = df.drop(columns=['A+', 'T+', 'N+'])
y = df['N+']

train_acc = []
train_pre = []
train_rec = []
train_f1 = []
train_auroc = []

test_acc = []
test_pre = []
test_rec = []
test_f1 = []
test_auroc = []

for i in range(0, 3):
    rs = np.random.randint(0, 75)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=rs)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device).unsqueeze(1)
    y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device).unsqueeze(1)

    model = SVM(X_train, y_train, kernel=gpytorch.kernels.RBFKernel()).to(device)
    bhl = torchmetrics.classification.BinaryHingeLoss(squared=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-4)

    # Training
    for epoch in range(1, 101):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = bhl(output, y_train)
        loss.backward()
        optimizer.step()

    # EVAL: Train
    model.eval()
    with torch.no_grad():
        y_pred_train = model(X_train)
        y_pred_train_labels = (y_pred_train > 0.5).float()
        y_pred_train_np = y_pred_train_labels.cpu().numpy()
        y_train_np = y_train.cpu().numpy()
        y_pred_train_proba = y_pred_train.cpu().numpy()

        train_acc.append(accuracy_score(y_train_np, y_pred_train_np))
        train_pre.append(precision_score(y_train_np, y_pred_train_np))
        train_rec.append(recall_score(y_train_np, y_pred_train_np))
        train_f1.append(f1_score(y_train_np, y_pred_train_np))
        train_auroc.append(roc_auc_score(y_train_np, y_pred_train_proba))

    # EVAL: Test
    with torch.no_grad():
        y_pred_test = model(X_test)
        y_pred_test_labels = (y_pred_test > 0.5).float()
        y_pred_test_np = y_pred_test_labels.cpu().numpy()
        y_test_np = y_test.cpu().numpy()
        y_pred_test_proba = y_pred_test.cpu().numpy()

        test_acc.append(accuracy_score(y_test_np, y_pred_test_np))
        test_pre.append(precision_score(y_test_np, y_pred_test_np))
        test_rec.append(recall_score(y_test_np, y_pred_test_np))
        test_f1.append(f1_score(y_test_np, y_pred_test_np))
        test_auroc.append(roc_auc_score(y_test_np, y_pred_test_proba))

print("=== Train Metrics ===")
print("Accuracy: ", np.mean(train_acc), "±", np.std(train_acc))
print("Precision:", np.mean(train_pre), "±", np.std(train_pre))
print("Recall:   ", np.mean(train_rec), "±", np.std(train_rec))
print("F1 Score: ", np.mean(train_f1), "±", np.std(train_f1))
print("AUROC:    ", np.mean(train_auroc), "±", np.std(train_auroc))

print("\n=== Test Metrics ===")
print("Accuracy: ", np.mean(test_acc), "±", np.std(test_acc))
print("Precision:", np.mean(test_pre), "±", np.std(test_pre))
print("Recall:   ", np.mean(test_rec), "±", np.std(test_rec))
print("F1 Score: ", np.mean(test_f1), "±", np.std(test_f1))
print("AUROC:    ", np.mean(test_auroc), "±", np.std(test_auroc))

=== Train Metrics ===
Accuracy:  0.906201550387597 ± 0.017124590724321376
Precision: 0.9006516924072007 ± 0.01514374518513446
Recall:    0.8930752524988227 ± 0.02620045186961263
F1 Score:  0.8968100767575832 ± 0.020685453788066428
AUROC:     0.9475268312342742 ± 0.00498663624911052

=== Test Metrics ===
Accuracy:  0.8641975308641975 ± 0.023096650535641576
Precision: 0.8695577391902605 ± 0.07032176204193617
Recall:    0.8537486323200608 ± 0.03149826110403518
F1 Score:  0.8585547426517777 ± 0.020439218877018543
AUROC:     0.9086917725968733 ± 0.025147133463591206


# MTL

In [52]:
class MultitaskSVM(torch.nn.Module):
    def __init__(self, features, targets, kernel: gpytorch.kernels.Kernel):
        super(MultitaskSVM, self).__init__()
        self.features = features
        self.targets = targets
        self.kernel = kernel
        self.w = torch.nn.Parameter(torch.randn(features.shape[1], 3), requires_grad=True)
        self.b = torch.nn.Parameter(torch.randn(1), requires_grad=True)
        self.classifier = torch.nn.Sigmoid()
        
    def forward(self, x):
        plane_output = x.matmul(self.w) + self.b
        return self.classifier(plane_output)

In [92]:
X = df.drop(columns=['A+', 'T+', 'N+'])
y = df.filter(['A+', 'T+', 'N+'])

train_acc, train_pre, train_rec, train_f1, train_auroc = [], [], [], [], []
test_acc, test_pre, test_rec, test_f1, test_auroc = [], [], [], [], []

X = df.drop(columns=['A+', 'T+', 'N+'])
y = df[['A+', 'T+', 'N+']]

for i in range(3):
    rs = np.random.randint(0, 75)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=rs)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

    model = MultitaskSVM(X_train, y_train, kernel=gpytorch.kernels.RBFKernel()).to(device)
    bhl = torchmetrics.classification.BinaryHingeLoss(squared=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.061911627281932204, weight_decay=8.933152146862321e-05)

    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = bhl(output, y_train)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        y_pred_train_all = model(X_train)
        y_pred_train_prob = torch.sigmoid(y_pred_train_all[:, 0])
        y_pred_train_labels = (y_pred_train_prob > 0.5).float()

        y_train_np = y_train[:, 0].cpu().numpy()
        y_pred_train_np = y_pred_train_labels.cpu().numpy()
        y_pred_train_proba = y_pred_train_prob.cpu().numpy()

        train_acc.append(accuracy_score(y_train_np, y_pred_train_np))
        train_pre.append(precision_score(y_train_np, y_pred_train_np))
        train_rec.append(recall_score(y_train_np, y_pred_train_np))
        train_f1.append(f1_score(y_train_np, y_pred_train_np))
        train_auroc.append(roc_auc_score(y_train_np, y_pred_train_proba))

    with torch.no_grad():
        y_pred_test_all = model(X_test)
        y_pred_test_prob = torch.sigmoid(y_pred_test_all[:, 0])
        y_pred_test_labels = (y_pred_test_prob > 0.5).float()

        y_test_np = y_test[:, 0].cpu().numpy()
        y_pred_test_np = y_pred_test_labels.cpu().numpy()
        y_pred_test_proba = y_pred_test_prob.cpu().numpy()

        test_acc.append(accuracy_score(y_test_np, y_pred_test_np))
        test_pre.append(precision_score(y_test_np, y_pred_test_np))
        test_rec.append(recall_score(y_test_np, y_pred_test_np))
        test_f1.append(f1_score(y_test_np, y_pred_test_np))
        test_auroc.append(roc_auc_score(y_test_np, y_pred_test_proba))

print("=== Train Metrics (A+ only) ===")
print(f"Accuracy:  {np.mean(train_acc)} ± {np.std(train_acc)}")
print(f"Precision: {np.mean(train_pre)} ± {np.std(train_pre)}")
print(f"Recall:    {np.mean(train_rec)} ± {np.std(train_rec)}")
print(f"F1 Score:  {np.mean(train_f1)} ± {np.std(train_f1)}")
print(f"AUROC:     {np.mean(train_auroc)} ± {np.std(train_auroc)}")

print("\n=== Test Metrics (A+ only) ===")
print(f"Accuracy:  {np.mean(test_acc)} ± {np.std(test_acc)}")
print(f"Precision: {np.mean(test_pre)} ± {np.std(test_pre)}")
print(f"Recall:    {np.mean(test_rec)} ± {np.std(test_rec)}")
print(f"F1 Score:  {np.mean(test_f1)} ± {np.std(test_f1)}")
print(f"AUROC:     {np.mean(test_auroc)} ± {np.std(test_auroc)}")



=== Train Metrics (A+ only) ===
Accuracy:  0.8310077519379844 ± 0.00897351697890715
Precision: 0.7982129416934667 ± 0.006184059800785192
Recall:    0.8766647506844172 ± 0.007878076379859105
F1 Score:  0.8355964812017436 ± 0.006685244658492608
AUROC:     0.8739915532164341 ± 0.003912586734090038

=== Test Metrics (A+ only) ===
Accuracy:  0.8425925925925926 ± 0.026189140043946214
Precision: 0.8249484878697242 ± 0.020051163941126878
Recall:    0.8803034895233477 ± 0.009786796772523988
F1 Score:  0.851660215221934 ± 0.014459247207377107
AUROC:     0.867645058046015 ± 0.013579797395908493


In [95]:
X = df.drop(columns=['A+', 'T+', 'N+'])
y = df.filter(['A+', 'T+', 'N+'])

train_acc, train_pre, train_rec, train_f1, train_auroc = [], [], [], [], []
test_acc, test_pre, test_rec, test_f1, test_auroc = [], [], [], [], []

X = df.drop(columns=['A+', 'T+', 'N+'])
y = df[['A+', 'T+', 'N+']]

for i in range(3):
    rs = np.random.randint(0, 75)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=rs)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

    model = MultitaskSVM(X_train, y_train, kernel=gpytorch.kernels.RBFKernel()).to(device)
    bhl = torchmetrics.classification.BinaryHingeLoss(squared=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.061911627281932204, weight_decay=8.933152146862321e-05)

    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = bhl(output, y_train)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        y_pred_train_all = model(X_train)
        y_pred_train_prob = torch.sigmoid(y_pred_train_all[:, 1])
        y_pred_train_labels = (y_pred_train_prob > 0.5).float()

        y_train_np = y_train[:, 0].cpu().numpy()
        y_pred_train_np = y_pred_train_labels.cpu().numpy()
        y_pred_train_proba = y_pred_train_prob.cpu().numpy()

        train_acc.append(accuracy_score(y_train_np, y_pred_train_np))
        train_pre.append(precision_score(y_train_np, y_pred_train_np))
        train_rec.append(recall_score(y_train_np, y_pred_train_np))
        train_f1.append(f1_score(y_train_np, y_pred_train_np))
        train_auroc.append(roc_auc_score(y_train_np, y_pred_train_proba))

    with torch.no_grad():
        y_pred_test_all = model(X_test)
        y_pred_test_prob = torch.sigmoid(y_pred_test_all[:, 1])
        y_pred_test_labels = (y_pred_test_prob > 0.5).float()

        y_test_np = y_test[:, 0].cpu().numpy()
        y_pred_test_np = y_pred_test_labels.cpu().numpy()
        y_pred_test_proba = y_pred_test_prob.cpu().numpy()

        test_acc.append(accuracy_score(y_test_np, y_pred_test_np))
        test_pre.append(precision_score(y_test_np, y_pred_test_np))
        test_rec.append(recall_score(y_test_np, y_pred_test_np))
        test_f1.append(f1_score(y_test_np, y_pred_test_np))
        test_auroc.append(roc_auc_score(y_test_np, y_pred_test_proba))

print("=== Train Metrics (T+ only) ===")
print(f"Accuracy:  {np.mean(train_acc)} ± {np.std(train_acc)}")
print(f"Precision: {np.mean(train_pre)} ± {np.std(train_pre)}")
print(f"Recall:    {np.mean(train_rec)} ± {np.std(train_rec)}")
print(f"F1 Score:  {np.mean(train_f1)} ± {np.std(train_f1)}")
print(f"AUROC:     {np.mean(train_auroc)} ± {np.std(train_auroc)}")

print("\n=== Test Metrics (T+ only) ===")
print(f"Accuracy:  {np.mean(test_acc)} ± {np.std(test_acc)}")
print(f"Precision: {np.mean(test_pre)} ± {np.std(test_pre)}")
print(f"Recall:    {np.mean(test_rec)} ± {np.std(test_rec)}")
print(f"F1 Score:  {np.mean(test_f1)} ± {np.std(test_f1)}")
print(f"AUROC:     {np.mean(test_auroc)} ± {np.std(test_auroc)}")



=== Train Metrics (T+ only) ===
Accuracy:  0.841860465116279 ± 0.006846326252967328
Precision: 0.8106826677159158 ± 0.010648550076435847
Recall:    0.8804678016446493 ± 0.004302283007272975
F1 Score:  0.8440792275198916 ± 0.005032863087160245
AUROC:     0.8733227503205442 ± 0.006770690828057027

=== Test Metrics (T+ only) ===
Accuracy:  0.8487654320987654 ± 0.028622279307085476
Precision: 0.8295734342017652 ± 0.018218613712946644
Recall:    0.9013360646657477 ± 0.026152815149260034
F1 Score:  0.8637687069934068 ± 0.017957695892248854
AUROC:     0.8769443876026329 ± 0.025926167347265876


In [98]:
X = df.drop(columns=['A+', 'T+', 'N+'])
y = df.filter(['A+', 'T+', 'N+'])

train_acc, train_pre, train_rec, train_f1, train_auroc = [], [], [], [], []
test_acc, test_pre, test_rec, test_f1, test_auroc = [], [], [], [], []

X = df.drop(columns=['A+', 'T+', 'N+'])
y = df[['A+', 'T+', 'N+']]

for i in range(3):
    rs = np.random.randint(0, 75)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=rs)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
    y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

    model = MultitaskSVM(X_train, y_train, kernel=gpytorch.kernels.RBFKernel()).to(device)
    bhl = torchmetrics.classification.BinaryHingeLoss(squared=True).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.061911627281932204, weight_decay=8.933152146862321e-05)

    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = bhl(output, y_train)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        y_pred_train_all = model(X_train)
        y_pred_train_prob = torch.sigmoid(y_pred_train_all[:, 2])
        y_pred_train_labels = (y_pred_train_prob > 0.5).float()

        y_train_np = y_train[:, 0].cpu().numpy()
        y_pred_train_np = y_pred_train_labels.cpu().numpy()
        y_pred_train_proba = y_pred_train_prob.cpu().numpy()

        train_acc.append(accuracy_score(y_train_np, y_pred_train_np))
        train_pre.append(precision_score(y_train_np, y_pred_train_np))
        train_rec.append(recall_score(y_train_np, y_pred_train_np))
        train_f1.append(f1_score(y_train_np, y_pred_train_np))
        train_auroc.append(roc_auc_score(y_train_np, y_pred_train_proba))

    with torch.no_grad():
        y_pred_test_all = model(X_test)
        y_pred_test_prob = torch.sigmoid(y_pred_test_all[:, 2])
        y_pred_test_labels = (y_pred_test_prob > 0.5).float()

        y_test_np = y_test[:, 0].cpu().numpy()
        y_pred_test_np = y_pred_test_labels.cpu().numpy()
        y_pred_test_proba = y_pred_test_prob.cpu().numpy()

        test_acc.append(accuracy_score(y_test_np, y_pred_test_np))
        test_pre.append(precision_score(y_test_np, y_pred_test_np))
        test_rec.append(recall_score(y_test_np, y_pred_test_np))
        test_f1.append(f1_score(y_test_np, y_pred_test_np))
        test_auroc.append(roc_auc_score(y_test_np, y_pred_test_proba))

print("=== Train Metrics (N+ only) ===")
print(f"Accuracy:  {np.mean(train_acc)} ± {np.std(train_acc)}")
print(f"Precision: {np.mean(train_pre)} ± {np.std(train_pre)}")
print(f"Recall:    {np.mean(train_rec)} ± {np.std(train_rec)}")
print(f"F1 Score:  {np.mean(train_f1)} ± {np.std(train_f1)}")
print(f"AUROC:     {np.mean(train_auroc)} ± {np.std(train_auroc)}")

print("\n=== Test Metrics (N+ only) ===")
print(f"Accuracy:  {np.mean(test_acc)} ± {np.std(test_acc)}")
print(f"Precision: {np.mean(test_pre)} ± {np.std(test_pre)}")
print(f"Recall:    {np.mean(test_rec)} ± {np.std(test_rec)}")
print(f"F1 Score:  {np.mean(test_f1)} ± {np.std(test_f1)}")
print(f"AUROC:     {np.mean(test_auroc)} ± {np.std(test_auroc)}")



=== Train Metrics (N+ only) ===
Accuracy:  0.8248062015503876 ± 0.016149353996898954
Precision: 0.798026448325551 ± 0.02271156721922992
Recall:    0.8671624764161292 ± 0.011324381340577012
F1 Score:  0.8309697479419936 ± 0.013757553415264058
AUROC:     0.8660051403864929 ± 0.007577443607398803

=== Test Metrics (N+ only) ===
Accuracy:  0.8549382716049383 ± 0.008729713347982107
Precision: 0.8110100860461958 ± 0.0426575846051556
Recall:    0.9244073536526368 ± 0.05542254050543656
F1 Score:  0.8612836438923396 ± 0.005855956780012828
AUROC:     0.895397801883651 ± 0.02053242050444273
